In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('datasimple.csv')

In [3]:
df['daily_diff']= df['close'] - df['open']
#calculate daily return
df['target'] =  np.where((df['daily_diff'] > 0),
                         (((df['close'] / df['open']) - 1) * 100),
                         (((df['open'] - df['close']) / df['open']) * (-100)))

In [4]:
df.shape

(392, 10)

In [5]:
df.head()

,time,open,high,low,close,KAMA,Volume,Volume MA,daily_diff,target
0,2020-11-11T02:00:00+02:00,450.34,476.25,449.28,463.09,409.605256,1.205783e+06,891616.690705,12.75,2.831194
1,2020-11-12T02:00:00+02:00,463.09,470.00,451.20,462.39,411.539989,9.907766e+05,904763.906041,-0.70,-0.151159
2,2020-11-13T02:00:00+02:00,462.48,478.01,457.12,476.43,415.100806,9.766651e+05,932635.317364,13.95,3.016347
3,2020-11-14T02:00:00+02:00,476.42,477.47,452.00,460.89,416.605915,7.352528e+05,948618.607829,-15.53,-3.259729
4,2020-11-15T02:00:00+02:00,460.90,462.89,440.19,448.08,417.443094,6.652176e+05,937122.352532,-12.82,-2.781514


In [6]:
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, colname = 'var'):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s%d(t-%d)' % (colname, j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s%d(t)' % (colname,j+1)) for j in range(n_vars)]
        else:
            names += [('%s%d(t+%d)' % (colname,j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [7]:
def get_data(df,cols,t_num):
    df_list = list()
    for i in cols:
        df_list.append(series_to_supervised(df[i].to_list(),colname=i,n_in=t_num))
    df_list.append(df['target'].iloc[t_num:])
    new_df = pd.concat(df_list, axis=1)
    new_df['target_flag'] = np.where(new_df['target'] > 0, 'Buy', 'Sell')
    return new_df

In [8]:
cols=['open','high','low']

In [9]:
new_df=get_data(df,cols,3)

In [10]:
new_df

,open1(t-3),open1(t-2),open1(t-1),open1(t),high1(t-3),high1(t-2),high1(t-1),high1(t),low1(t-3),low1(t-2),low1(t-1),low1(t),target,target_flag
3,450.34,463.09,462.48,476.42,476.25,470.00,478.01,477.47,449.28,451.20,457.12,452.00,-3.259729,Sell
4,463.09,462.48,476.42,460.90,470.00,478.01,477.47,462.89,451.20,457.12,452.00,440.19,-2.781514,Sell
5,462.48,476.42,460.90,448.07,478.01,477.47,462.89,466.00,457.12,452.00,440.19,445.45,2.785279,Buy
6,476.42,460.90,448.07,460.51,477.47,462.89,466.00,484.88,452.00,440.19,445.45,459.80,4.699138,Buy
7,460.90,448.07,460.51,482.11,462.89,466.00,484.88,495.31,440.19,445.45,459.80,456.60,-0.767460,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,4444.54,4630.25,4582.95,4511.35,4753.00,4778.75,4634.83,4654.88,4346.36,4523.99,4433.00,4032.50,-6.552806,Sell
388,4630.25,4582.95,4511.35,4215.01,4778.75,4634.83,4654.88,4238.59,4523.99,4433.00,4032.50,3503.68,-2.319330,Sell
389,4582.95,4511.35,4215.01,4117.25,4634.83,4654.88,4238.59,4248.19,4433.00,4032.50,3503.68,4031.39,1.923371,Buy
390,4511.35,4215.01,4117.25,4196.44,4654.88,4238.59,4248.19,4376.15,4032.50,3503.68,4031.39,3921.00,3.601863,Buy


In [11]:
from pycaret.classification import *
s = setup(new_df,
          target = 'target_flag',
          ignore_features = ['target'],
          fold_strategy = 'timeseries',
          feature_ratio =True,
          normalize=True,
          remove_outliers = True,
          data_split_shuffle=False,  
          outliers_threshold = 0.01)

,Description,Value
0,session_id,168
1,Target,target_flag
2,Target Type,Binary
3,Label Encoded,"Buy: 0, Sell: 1"
4,Original Data,"(389, 14)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [12]:
best=compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7500,0.7427,0.5797,0.6677,0.6121,0.4357,0.4444,0.0950
lightgbm,Light Gradient Boosting Machine,0.7250,0.7257,0.5700,0.6191,0.5793,0.3708,0.3851,0.0180
qda,Quadratic Discriminant Analysis,0.7042,0.7323,0.7052,0.5861,0.6222,0.3790,0.3972,0.0070
ridge,Ridge Classifier,0.6833,0.0000,0.4500,0.7030,0.4999,0.3300,0.3681,0.0060
lr,Logistic Regression,0.6708,0.6914,0.3624,0.6139,0.4329,0.2613,0.2829,0.4940
lda,Linear Discriminant Analysis,0.6708,0.7047,0.5163,0.6893,0.5499,0.3205,0.3482,0.0050
gbc,Gradient Boosting Classifier,0.6458,0.7196,0.6164,0.5946,0.5709,0.2938,0.3155,0.0340
et,Extra Trees Classifier,0.6125,0.7011,0.3188,0.4902,0.3461,0.1229,0.1354,0.0830
knn,K Neighbors Classifier,0.6083,0.5566,0.2074,0.4867,0.2595,0.0755,0.1072,0.0120
dummy,Dummy Classifier,0.6000,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0050


In [13]:
best

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=168, verbose=0,
                       warm_start=False)

In [ ]:
evaluate_model(best)

In [14]:
pre=predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.6838,0.7371,0.4464,0.8065,0.5747,0.3546,0.3940


In [15]:
pre['res']=np.where(pre['target_flag']==pre['Label'],1,0)

In [16]:
pre

,open1(t-3),open1(t),high1(t-2),high1(t-1),low1(t-3),low1(t-2),low1(t-1),low1(t),open1(t-3)_divide_open1(t),low1(t)_divide_open1(t),high1(t-2)_divide_high1(t-1),target_flag,Label,Score,res
0,1.609328,1.438523,1.553511,1.502819,1.695843,1.770761,1.577692,1.635750,1.118736,1.137104,1.033732,Buy,Buy,0.73,1
1,1.586198,1.768659,1.513726,1.603848,1.783680,1.591500,1.646945,1.846977,0.896837,1.044281,0.943809,Sell,Sell,0.51,1
2,1.609196,1.697939,1.614701,1.607430,1.604343,1.660811,1.858097,1.734030,0.947735,1.021256,1.004523,Buy,Sell,0.77,0
3,1.472722,1.750755,1.618281,1.594943,1.673684,1.872141,1.745190,1.764647,0.841193,1.007935,1.014633,Sell,Sell,0.84,1
4,1.802559,1.559321,1.605800,1.612003,1.885103,1.759139,1.775796,1.582485,1.155990,1.014855,0.996152,Sell,Sell,0.68,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,3.142600,3.189763,3.263991,3.090343,3.343361,3.555646,3.424583,2.905020,0.985214,0.910732,1.056190,Sell,Sell,0.62,1
113,3.364358,2.835580,3.100394,3.113146,3.569321,3.439949,2.915761,2.232931,1.186480,0.787469,0.995904,Sell,Sell,0.75,1
114,3.307877,2.718739,3.123185,2.639686,3.453574,2.930697,2.243912,2.903609,1.216695,1.067999,1.183165,Buy,Buy,0.65,1
115,3.222378,2.813385,2.649980,2.650604,2.944107,2.258282,2.914351,2.763312,1.145374,0.982202,0.999764,Buy,Buy,0.57,1


In [20]:
preds=new_df[-117:]

In [26]:
preds['Sc']=np.array(pre['Score'])
preds['Res']=np.array(pre['res'])
preds['Label']=np.array(pre['Label'])

In [53]:
last=preds[['Label','target_flag','target']]

In [55]:
last.loc[(last['Label']=='Buy')&(last['target_flag']=='Buy'),'Result']=1
last.loc[(last['Label']=='Sell')&(last['target_flag']=='Sell'),'Result']=0
last.Result.fillna(2,inplace=True)

In [60]:
last

,Label,target_flag,target,Result
275,Buy,Buy,9.067631,1.0
276,Sell,Sell,-1.780979,0.0
277,Sell,Buy,1.346241,2.0
278,Sell,Sell,-4.862503,0.0
279,Sell,Sell,-4.304171,0.0
...,...,...,...,...
387,Sell,Sell,-6.552806,0.0
388,Sell,Sell,-2.319330,0.0
389,Buy,Buy,1.923371,1.0
390,Buy,Buy,3.601863,1.0
